In [1]:
import os 
os.chdir('../../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
import utils.interpretable_functions as interpret
import utils.RiskSLIM as slim
import utils.stumps as stumps
import utils.fairness_functions as fairness
import utils.risktool as risktool
from utils.load_settings import load_settings

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import roc_auc_score

from pprint import pprint
from riskslim.helper_functions import load_data_from_csv, print_model

# restore saved variables
%store -r summary_drug2_KY_interpret

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### CART & EBM

In [2]:
### load data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-data/kentucky_data.csv").sort_values('person_id')
x = data.loc[:,:'current_violence20'].drop(['arnold_fta_raw', 'arnold_nca_raw', 'arnold_nvca_raw'], axis=1)
y = data['drug_two_year'].values

In [3]:
#### CART
depth = [5,6,7,8,9,10]
cart_summary = interpret.CART(X=x,
                         Y=y,
                         depth=depth,
                         seed=816)

#### EBM
estimators = [40]
depth = [2]
learning_rate = [0.3]
ebm_summary = interpret.EBM(X=x,
                       Y=y,
                       learning_rate=learning_rate,
                       depth=depth,
                       estimators=estimators,
                       seed =816)

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:44: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:44: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

i

In [4]:
np.mean(cart_summary['holdout_test_auc']), np.mean(cart_summary['auc_diffs'])

(0.7290215063502338, 0.010609467431846587)

In [5]:
np.mean(ebm_summary['holdout_test_auc']), np.mean(ebm_summary['auc_diffs'])

(0.732538168965449, 0.004428045998580443)

### Lasso Stumps

In [6]:
## load stumps & original data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-data/kentucky_stumps.csv").sort_values('person_id')
original_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-data/kentucky_data.csv").sort_values('person_id')
original_data = original_data.loc[:, ['person_id', 'age_at_current_charge', 'p_charges']]
data = pd.merge(original_data, data, on='person_id')

## split X & Y
X_stumps = data.loc[:,:'current_violence201'].copy()
Y_stumps = data['drug_two_year'].values.copy()
Y_stumps[Y_stumps == -1] = 0
cols = X_stumps.columns[5:]

## load train & test stumps data
train_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-data/kentucky_train_stumps.csv").sort_values('person_id')
test_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-data/kentucky_test_stumps.csv").sort_values('person_id')
X_train_stumps = train_stumps.loc[:,:'current_violence201'].copy()
X_test_stumps = test_stumps.loc[:,:'current_violence201'].copy()
Y_train_stumps = train_stumps['drug_two_year'].values.copy()
Y_test_stumps = test_stumps['drug_two_year'].values.copy()
Y_train_stumps[Y_train_stumps == -1] = 0
Y_test_stumps[Y_test_stumps == -1] = 0

##### Single Stump Model

In [7]:
single_stump_model = stumps.stump_model(X_train_stumps, 
                                        Y_train_stumps, 
                                        X_test_stumps, 
                                        Y_test_stumps, 
                                        c=0.003, 
                                        columns=cols, 
                                        seed=816)
len(single_stump_model['features'])

29

In [8]:
single_stump_model['features']

['sex1',
 'age_at_current_charge19',
 'age_at_current_charge20',
 'age_at_current_charge24',
 'age_at_current_charge35',
 'age_at_current_charge39',
 'age_at_current_charge43',
 'age_at_current_charge47',
 'p_arrest2',
 'p_arrest3',
 'p_arrest4',
 'p_arrest5',
 'p_felony1',
 'p_felony2',
 'p_felony3',
 'p_felony4',
 'p_misdemeanor4',
 'p_drug1',
 'p_drug2',
 'p_drug3',
 'ADE1',
 'p_fta_two_year1',
 'p_fta_two_year2',
 'p_pending_charge1',
 'p_probation1',
 'p_incarceration1',
 'six_month1',
 'one_year1',
 'current_pending_charge1']

#### Nested Cross Validation

In [9]:
stump_summary = stumps.stump_cv(X = X_stumps, 
                                Y = Y_stumps, 
                                columns=cols, 
                                c_grid={'C': [0.001, 0.002, 0.003]}, 
                                seed = 816)

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

invalid value encountered in longlong_scalars



In [10]:
stump_summary['best_params'], np.mean(stump_summary['holdout_test_auc']), np.mean(stump_summary['auc_diffs'])

([{'C': 0.003}, {'C': 0.003}, {'C': 0.003}, {'C': 0.003}, {'C': 0.003}],
 0.7305119139694728,
 0.000676142389347123)

### RiskSLIM

In [27]:
## load stumps data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-data/kentucky_stumps.csv").sort_values('person_id')

## single stump model
single_stump_model = stumps.stump_model(X_train_stumps, 
                                        Y_train_stumps, 
                                        X_test_stumps, 
                                        Y_test_stumps, 
                                        c=0.0014,
                                        columns=cols, 
                                        seed=816)
len(single_stump_model['features'])

18

In [29]:
### Subset features
selected_features = single_stump_model['features']
if 'sex1' in selected_features:
    selected_features = ['drug_two_year', 'person_id', 'screening_date', 'race'] + selected_features
    indicator = 1
else:
    selected_features = ['drug_two_year', 'person_id', 'screening_date', 'race', 'sex1'] + selected_features
    indicator = 0

sub_data = data[selected_features]
sub_data = pd.merge(sub_data, original_data, on='person_id')
sub_X, sub_Y = sub_data.iloc[:,1:], sub_data.iloc[:,0].values
sub_X.insert(0, '(Intercept)', 1)

In [31]:
riskslim_summary = slim.risk_nested_cv(X=sub_X, 
                                       Y=sub_Y, 
                                       indicator = indicator,
                                       y_label='drug_two_year', 
                                       max_coef=5, 
                                       max_coef_number=5, 
                                       max_runtime=1000,
                                       max_offset=100,
                                       c=1e-6, 
                                       seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
02/12/20 @ 12:26 PM | 233 rows in lookup table
02/12/20 @ 12:26 PM | ------------------------------------------------------------
02/12/20 @ 12:26 PM | runnning initialization procedure
02/12/20 @ 12:26 PM | ------------------------------------------------------------
02/12/20 @ 12:26 PM | CPA produced 2 cuts
02/12/20 @ 12:26 PM | running naive rounding on 4 solutions
02/12/20 @ 12:26 PM | best objective value: 0.6931
02/12/20 @ 12:26 PM | rounding produced 4 integer solutions
02/12/20 @ 12:26 PM | best objective value is 0.6931
02/12/20 @ 12:26 PM | running sequential rounding on 4 solutions
02/12/20 @ 12:26 PM | best objective value: 0.6931
02/12/20 @ 12:26 PM | sequential rounding produced 0 integer solutions
02/12/20 @ 12:26 PM | polishing 4 solutions
02/12/20 @ 12:26 PM | best objective value: 0.6931
02/12/20 @ 12:26 PM | polishing produced 4 integer solutions
02/12/20 @ 12:26 PM | initialization produced 7 feasible soluti

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/12/20 @ 12:26 PM | adding 265 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2902.
Tried aggregator 1 time.
Reduced MIP has 38 rows, 40 columns, and 111 nonzeros.
Reduced MIP has 18 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

Elapsed time = 149.41 sec. (129347.16 ticks, tree = 13.44 MB, solutions = 16)
 152807 32625        0.2838    21        0.2843        0.2782  1241581    2.17%           rho_1 N 152807  61508     24
 156000 32499        0.2816    13        0.2843        0.2784  1261142    2.09%           rho_9 D 156000 155999     21
 159175 32289        cutoff              0.2843        0.2786  1280078    2.02%           rho_0 U 159175 159173     31
 162248 31920        0.2834    11        0.2843        0.2788  1299118    1.95%           rho_5 U 162248 162247     34
 165361 31575        cutoff              0.2843        0.2790  1317506    1.89%        alpha_15 U 165361 165359     37
 168522 31280        0.2810     8        0.2843        0.2792  1335640    1.82%           rho_8 U 168522  84389     33
 171569 30782        cutoff              0.2843        0.2793  1353402    1.75%        alpha_18 D 171569 171568     30
 174687 30363        0.2820     1        0.2843        0.2795  1370778    1.69%        al

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/12/20 @ 12:31 PM | adding 275 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2896.
Tried aggregator 1 time.
Reduced MIP has 38 rows, 40 columns, and 111 nonzeros.
Reduced MIP has 18 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

02/12/20 @ 12:33 PM | rounding produced 3 integer solutions
02/12/20 @ 12:33 PM | best objective value is 0.6931
02/12/20 @ 12:33 PM | running sequential rounding on 3 solutions
02/12/20 @ 12:33 PM | best objective value: 0.6931
02/12/20 @ 12:33 PM | sequential rounding produced 0 integer solutions
02/12/20 @ 12:33 PM | polishing 3 solutions
02/12/20 @ 12:33 PM | best objective value: 0.6931
02/12/20 @ 12:33 PM | polishing produced 3 integer solutions
02/12/20 @ 12:33 PM | initialization produced 5 feasible solutions
02/12/20 @ 12:33 PM | best objective value: 0.2902
02/12/20 @ 12:33 PM | ------------------------------------------------------------
02/12/20 @ 12:33 PM | completed initialization procedure
02/12/20 @ 12:33 PM | ------------------------------------------------------------
02/12/20 @ 12:33 PM | 233 rows in lookup table
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_Parallel                                1
CPX

...

 189788  7377        cutoff              0.2831        0.2822  1206854    0.33%           rho_5 U 189788  57538     27
 194731  4450        cutoff              0.2831        0.2825  1218342    0.21%         alpha_7 U 194731 194730     29
 199652  1339        cutoff              0.2831        0.2829  1228380    0.07%           rho_2 U 199652 158120     18

Gomory fractional cuts applied:  1
User cuts applied:  1473

Root node processing (before b&c):
  Real time             =    0.11 sec. (1.91 ticks)
Sequential b&c:
  Real time             =  139.52 sec. (157663.37 ticks)
                          ------------
Total (root+branch&cut) =  139.63 sec. (157665.28 ticks)
setting c0 = 0.0 to ensure that intercept is not penalized
02/12/20 @ 01:36 PM | 233 rows in lookup table
02/12/20 @ 01:36 PM | ------------------------------------------------------------
02/12/20 @ 01:36 PM | runnning initialization procedure
02/12/20 @ 01:36 PM | ----------------------------------------------------------

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/12/20 @ 01:36 PM | adding 268 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2901.
Tried aggregator 1 time.
Reduced MIP has 38 rows, 40 columns, and 111 nonzeros.
Reduced MIP has 18 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

 195250  6369        cutoff              0.2832        0.2822  1328128    0.33%        alpha_15 U 195250 195233     44
 200053  3275        cutoff              0.2832        0.2827  1341341    0.19%          rho_14 U 200053 156467     29

Gomory fractional cuts applied:  1
User cuts applied:  1327

Root node processing (before b&c):
  Real time             =    0.08 sec. (1.92 ticks)
Sequential b&c:
  Real time             =  139.73 sec. (152358.82 ticks)
                          ------------
Total (root+branch&cut) =  139.81 sec. (152360.74 ticks)
setting c0 = 0.0 to ensure that intercept is not penalized
02/12/20 @ 01:39 PM | 233 rows in lookup table
02/12/20 @ 01:39 PM | ------------------------------------------------------------
02/12/20 @ 01:39 PM | runnning initialization procedure
02/12/20 @ 01:39 PM | ------------------------------------------------------------
02/12/20 @ 01:39 PM | CPA produced 2 cuts
02/12/20 @ 01:39 PM | running naive rounding on 3 solutions
02/12/20 @ 01:

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/12/20 @ 01:39 PM | adding 277 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2901.
Tried aggregator 1 time.
Reduced MIP has 38 rows, 40 columns, and 111 nonzeros.
Reduced MIP has 18 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:44: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

invalid value encountered in longlong_scalars



In [33]:
np.mean(riskslim_summary['validation_auc']), np.mean(riskslim_summary['test_auc'])

(0.680577865740604, 0.6809500207799113)

#### Single RiskSLIM Model

In [34]:
selected_features = ['drug_two_year'] + single_stump_model['features']
sub_train_data = train_stumps[selected_features]
sub_test_data = test_stumps[selected_features]

## split x 
sub_train_X = sub_train_data.iloc[:,1:]
sub_train_X.insert(0, '(Intercept)', 1)
sub_cols = sub_train_X.columns.tolist()
sub_train_X = sub_train_X.values
sub_test_X = sub_test_data.iloc[:,1:].values

## split y
sub_train_Y = sub_train_data.iloc[:,0].values.reshape(-1,1)
sub_test_Y = sub_test_data.iloc[:,0].values.reshape(-1,1)

## sample weight
sample_weights = np.repeat(1, len(sub_train_Y))

In [35]:
new_train_data = {
    'X': sub_train_X,
    'Y': sub_train_Y,
    'variable_names': sub_cols,
    'outcome_name': 'drug_two_year',
    'sample_weights': sample_weights
}

In [36]:
model_info, mip_info, lcpa_info = slim.risk_slim(new_train_data, 
                                                 max_coefficient=5, 
                                                 max_L0_value=5, 
                                                 c0_value=1e-6, 
                                                 max_offset=100, 
                                                 max_runtime=1000)
print_model(model_info['solution'], new_train_data)

setting c0 = 0.0 to ensure that intercept is not penalized
02/12/20 @ 01:42 PM | 233 rows in lookup table
02/12/20 @ 01:43 PM | ------------------------------------------------------------
02/12/20 @ 01:43 PM | runnning initialization procedure
02/12/20 @ 01:43 PM | ------------------------------------------------------------
02/12/20 @ 01:43 PM | CPA produced 2 cuts
02/12/20 @ 01:43 PM | running naive rounding on 3 solutions
02/12/20 @ 01:43 PM | best objective value: 0.6931
02/12/20 @ 01:43 PM | rounding produced 3 integer solutions
02/12/20 @ 01:43 PM | best objective value is 0.6931
02/12/20 @ 01:43 PM | running sequential rounding on 3 solutions
02/12/20 @ 01:43 PM | best objective value: 0.6931
02/12/20 @ 01:43 PM | sequential rounding produced 0 integer solutions
02/12/20 @ 01:43 PM | polishing 3 solutions
02/12/20 @ 01:43 PM | best objective value: 0.6931
02/12/20 @ 01:43 PM | polishing produced 3 integer solutions
02/12/20 @ 01:43 PM | initialization produced 5 feasible soluti

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/12/20 @ 01:43 PM | adding 287 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2904.
Tried aggregator 1 time.
Reduced MIP has 38 rows, 40 columns, and 111 nonzeros.
Reduced MIP has 18 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

 237693 10264        cutoff              0.2834        0.2821  1554464    0.44%          rho_13 D 237693  56869     35
 242908  7003        cutoff              0.2834        0.2825  1571063    0.32%          rho_10 D 242908 140108     36
 247749  3101        0.2833    15        0.2834        0.2829  1585583    0.16%          rho_12 D 247749 222023     23

Gomory fractional cuts applied:  1
User cuts applied:  1175

Root node processing (before b&c):
  Real time             =    0.17 sec. (1.92 ticks)
Sequential b&c:
  Real time             =  157.64 sec. (163277.20 ticks)
                          ------------
Total (root+branch&cut) =  157.81 sec. (163279.12 ticks)
+----------------------------------------------+-------------------+-----------+
| Pr(Y = +1) = 1.0/(1.0 + exp(-(-3 + score))   |                   |           |
| ============================================ | ================= | ========= |
| p_arrest3                                    |          1 points |   + ..... |
|

In [37]:
sub_train_X = sub_train_X[:,1:]
sub_train_Y[sub_train_Y == -1] = 0
sub_test_Y[sub_test_Y == -1] = 0

In [38]:
roc_auc_score(sub_train_Y, slim.riskslim_prediction(sub_train_X, np.array(cols), model_info).reshape(-1,1))

0.6824424104857194

In [39]:
roc_auc_score(sub_test_Y, slim.riskslim_prediction(sub_test_X, np.array(cols), model_info).reshape(-1,1))

0.6673833309265927

### Arnold PSA

In [40]:
### load data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-data/kentucky_data.csv").sort_values('person_id')
settings = load_settings()
for decoder_name, decoder_dict in settings['decoders'].items():
    data = data.replace({decoder_name: decoder_dict})  
    
# compute scaled arnold scores
for score in ['arnold_nca' , 'arnold_nvca', 'arnold_fta']:
    data[score] = data[score + "_raw"]
    data = data.replace({score: settings[score + '_scaler'] })
    if score != 'arnold_nvca':
        data[score] = data[score].astype(int)
X_arnold = data.loc[:,['arnold_nca', 'sex', 'race', 'person_id', 'screening_date', 'age_at_current_charge', 'p_charges']]
Y_arnold = data['drug_two_year'].values

In [41]:
arnold = risktool.risktool(X_arnold, Y_arnold, label='arnold_nca')

In [42]:
np.mean(arnold['auc'])

0.6717414339141878

#### Single Arnold PSA

In [43]:
### load data
test_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-data/kentucky_test.csv").sort_values('person_id')
settings = load_settings()
for decoder_name, decoder_dict in settings['decoders'].items():
    test_data = test_data.replace({decoder_name: decoder_dict})  
    
# compute scaled arnold scores
for score in ['arnold_nca' , 'arnold_nvca', 'arnold_fta']:
    test_data[score] = test_data[score + "_raw"]
    test_data = test_data.replace({score: settings[score + '_scaler'] })
    if score != 'arnold_nvca':
        test_data[score] = test_data[score].astype(int)

In [44]:
X = test_data['arnold_nca'].values
Y = test_data['drug_two_year'].values
roc_auc_score(Y, X)

0.6584418006181798

### Results

In [45]:
#### save results
summary_drug2_KY_interpret = {"cart": cart_summary,
                              "ebm": ebm_summary, 
                              'stumps': stump_summary, 
                              'riskslim': riskslim_summary, 
                              'arnold': arnold}
%store summary_drug2_KY_interpret

Stored 'summary_drug2_KY_interpret' (dict)


In [46]:
results = [["cart", np.mean(cart_summary['holdout_test_auc']), np.mean(cart_summary['auc_diffs'])],
           ["ebm", np.mean(ebm_summary['holdout_test_auc']), np.mean(ebm_summary['auc_diffs'])], 
           ["stumps", np.mean(stump_summary['holdout_test_auc']), np.mean(stump_summary['auc_diffs'])],
           ['riskslim', np.mean(riskslim_summary['test_auc'])],
           ['arnold', np.mean(arnold['auc'])]]
results

[['cart', 0.7290215063502338, 0.010609467431846587],
 ['ebm', 0.732538168965449, 0.004428045998580443],
 ['stumps', 0.7305119139694728, 0.000676142389347123],
 ['riskslim', 0.6809500207799113],
 ['arnold', 0.6717414339141878]]

In [49]:
auc = [np.mean(cart_summary['holdout_test_auc']), 
       np.mean(ebm_summary['holdout_test_auc']), 
       np.mean(stump_summary['holdout_test_auc']), 
       np.mean(riskslim_summary['test_auc'])]

In [50]:
path = "./kentucky/logs/interpretable/"
results = [["Drug", np.str((round(np.mean(cart_summary['holdout_test_auc']), 3))) + " (" + np.str(round(np.std(cart_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(ebm_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(ebm_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(stump_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(stump_summary['holdout_test_auc']), 3)) + ")",             
            np.str(round(np.mean(riskslim_summary['test_auc']),3)) + " (" + np.str(round(np.std(riskslim_summary['test_auc']), 3)) + ")", 
            round(np.max(auc) - np.min(auc), 3),
            np.str(round(np.mean(arnold['auc']), 3)) + " (" + np.str(round(np.std(arnold['auc']),3)) + ")"]]
with open(path + 'KY-two-year-interpretable-summary.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)

### Save Fairness Matrix

In [51]:
matrix = ['confusion_matrix_stats', 'calibration_stats', 'race_auc', 'condition_pn', 'no_condition_pn']
name = ['confusion', 'calibration', 'race-auc', 'condition-pn', 'no-condition-pn']

In [52]:
for i in range(len(matrix)):
    
    if ( (i == 0) | (i == 1) ):
        
        path = './kentucky/fairness/' + name[i] + '/two-year/drug/'
        ## confusion matrix and calibration only applies to cart, ebm, riskslim and stumps
        cart_matrix = cart_summary[matrix[i]]
        ebm_matrix = ebm_summary[matrix[i]]
        riskslim_matrix = riskslim_summary[matrix[i]]
        stumps_matrix = stump_summary[matrix[i]]
        
        ## save results
        cart_matrix.to_csv(path+'cart-'+name[i]+'.csv', index=None,header=True)
        ebm_matrix.to_csv(path+'ebm-'+name[i]+'.csv', index=None,header=True)
        riskslim_matrix.to_csv(path+'riskslim-'+name[i]+'.csv', index=None,header=True)
        stumps_matrix.to_csv(path+'stumps-'+name[i]+'.csv', index=None,header=True)
        
    else:
        path = './kentucky/fairness/' + name[i] + '/two-year/drug/'
        
        ## including arnold and compas now
        cart_matrix = cart_summary[matrix[i]]
        ebm_matrix = ebm_summary[matrix[i]]
        riskslim_matrix = riskslim_summary[matrix[i]]
        stumps_matrix = stump_summary[matrix[i]]
        arnold_matrix = arnold[matrix[i]]
        
        ## save results
        cart_matrix.to_csv(path+'cart-'+name[i]+'.csv', index=None,header=True)
        ebm_matrix.to_csv(path+'ebm-'+name[i]+'.csv', index=None,header=True)
        riskslim_matrix.to_csv(path+'riskslim-'+name[i]+'.csv', index=None,header=True)
        stumps_matrix.to_csv(path+'stumps-'+name[i]+'.csv', index=None,header=True)
        arnold_matrix.to_csv(path+'arnold-'+name[i]+'.csv', index=None,header=True)